In [4]:
%pylab inline
from sklearn import svm
import numpy as np
import pickle as pk
import matplotlib.pylab as plt


data_path = '/Users/ardavan/Documents/Research/piriform/data/sp042915a_for_SVM.dat'
mydict = pk.load(open(data_path, 'rb'))

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [69]:
#Parametrs of the pipeline
shuffling_seed = 2 #seed of the random shuffling
scale = True  #whether we want to scale the dataset
train_ratio = 0.7 # ratio of the training samples
validation_ratio = 0.15 #ratio of the validation examples
C_list = [0.0001, 0.001, 0.01, 0.1, 1, 2, 5, 10] #List of regularization parameters for SVM
gamma_list = [0, 1/500., 1/50., 1/5., 1] #List of gamma parameters for the RBF kernel in SVM

In [70]:
# Shuffle the rows of the data matrix (and also the target)
np.random.seed(shuffling_seed)
shuffled_indices = range(mydict['data'].shape[0])
np.random.shuffle(shuffled_indices)

#Shuffle the rows of the data matrix:
old_data = mydict['data']
new_data = old_data[np.array(shuffled_indices),:]

if scale == True:
#normalize the data
    new_data -= np.mean(new_data)
    new_data /= np.std(new_data)

#Shuffle the rows of the target values
old_target = mydict['target']
new_target = old_target[np.array(shuffled_indices)]


In [71]:
# Make train, validation and test datasets

#First find the rows
test_ratio = 1- train_ratio - validation_ratio
num_train_rows = int(train_ratio * new_data.shape[0])
num_validation_rows = int(validation_ratio * new_data.shape[0])
num_test_rows = new_data.shape[0] - num_train_rows - num_validation_rows
print 'total_num: '+ str(new_data.shape[0]) + '   num_train: '+ str(num_train_rows) +'    num_test: ' + str(num_test_rows) + '    num_validation: ' + str(num_validation_rows)

#Now make the three datasets
train_data_features = new_data[:num_train_rows, :]
train_data_labels = new_target[:num_train_rows]
validation_data_features = new_data[num_train_rows:num_train_rows+num_validation_rows+1, :]
validation_data_labels = new_target[num_train_rows:num_train_rows+num_validation_rows+1]
test_data_features = new_data[-num_test_rows+1:, :]
test_data_labels = new_target[-num_test_rows+1:]


total_num: 115   num_train: 80    num_test: 18    num_validation: 17


In [72]:
#Train the SVM and find the best setting over the validation set:

accuracy_dict = {}
for C in C_list:
    for gamma in gamma_list:
        clf = svm.SVC(C = C, gamma = gamma)
        clf.fit(train_data_features, train_data_labels)
        accuracy_dict[(C, gamma)] = clf.score(validation_data_features, validation_data_labels)

#return the best settin of the hyperparameters
all_good_settings = []
all_accuracy_vals = accuracy_dict.values()
max_accuracy = max(all_accuracy_vals)
for key, value in accuracy_dict.iteritems():
    if value == max_accuracy:
        all_good_settings.append(key)

#print the test accuracy for the best setting:
print 'All good settings (C, gamma) :  ' + str(all_good_settings)
clf = svm.SVC(C = all_good_settings[0][0], gamma = all_good_settings[0][1])
clf.fit(train_data_features, train_data_labels)
train_accuracy = clf.score(train_data_features, train_data_labels)
test_accuracy = clf.score(test_data_features, test_data_labels)
print ''
print 'Accuracy after choosing the first setting: '
print 'train_accuracy: ' + str(train_accuracy)
print 'test_accuracy: ' + str(test_accuracy)

All good settings (C, gamma) :  [(10, 0.002)]

Accuracy after choosing the first setting: 
train_accuracy: 0.625
test_accuracy: 0.0


In [ ]:
# Cross validation (built-in)


In [ ]:
# Give the cell numbers to remove and do the same analysis
#try cross validation